In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

/home/ubuntu/code/mistral-reasoning-ft/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    attn_implementation="flash_attention_2",
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [01:49<00:00, 36.66s/it]


In [9]:
def generate_response(prompt):
    # Format the input using the chat template
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}], return_tensors="pt"
    ).to(base_model.device)

    # Generate with parameters suited for reasoning
    outputs = base_model.generate(
        input_ids=inputs,
        max_new_tokens=32768,
        temperature=0.5,
        top_p=0.9,
        do_sample=True,
    )

    # Decode the response, removing the prompt
    response = tokenizer.decode(outputs[0][inputs.shape[1] :], skip_special_tokens=True)
    return response

In [10]:
question = "Consider the following two person game. A number of pebbles are situated on the table. Two players make their moves alternately. A move consists of taking off the table  $x$  pebbles where  $x$  is the square of any positive integer. The player who is unable to make a move loses. Prove that there are infinitely many initial situations in which the second player can win no matter how his opponent plays."
response = generate_response(question)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To prove that there are infinitely many initial situations where the second player can win, we can show that for any positive integer n, the second player can win if there are at least n^2 + 1 pebbles on the table initially.

We will use mathematical induction to prove this claim.

Base case: n = 1
If there are at least 2 pebbles on the table, the second player can take 1 pebble (1^2), leaving 1 pebble. The first player, in their move, can only take 1 pebble (1^2), leaving no pebbles. The second player wins.

Inductive step:
Assume the claim holds for some positive integer k, i.e., if there are at least (k+1)^2 + 1 pebbles on the table, the second player can win.

Now consider the case when there are at least (k+2)^2 + 1 pebbles on the table. The first player can take at most (1^2) = 1 pebble in their first move, leaving at least (k+2)^2 pebbles on the table.

The second player can take (k+1)^2 pebbles in their move, leaving at least 1 pebble on the table. The first player, in their ne